In [1]:
library(phytools)

Loading required package: ape

Loading required package: maps



In [2]:
buildCoev<-function(s,d,r,nStates=4,withinProfile=FALSE) {
    
    #s/d decrease if coevolution becomes stronger cf Dib et al 2014
    
    dim=nStates*nStates
    if(nStates == 4) {
        states<-c("A","C","G","T")
    }
    else if(nStates == 20) {
        states<-c("A","R","N","D","C","Q","E","G","H","I","L","K","M","F","P","S","T","W","Y","V")
    }
    else {
        states<-as.character(1:nStates)
    }
    
    dim.names<-paste(rep(states, each=nStates), rep(states, nStates), sep="")

    #set frequencies from a Dirichlet. The two states with highest frequencies are the profile
    f<-rgamma(dim, 1, 1)
    f<-f/sum(f)
    profile<-dim.names[order(f, decreasing=TRUE)[1:2]]
    while(adist(profile[1],profile[2])<2) {
        f<-rgamma(dim, 1, 1)
        f<-f/sum(f)
        profile<-dim.names[order(f, decreasing=TRUE)[1:2]]
    }
    
    freq<-matrix(0, dim, dim)
    rownames(freq)<-colnames(freq)<-dim.names
    
    diag(freq)<-f
    #diag(freq)<-rep(f.nonprofile, dim) #assume here equal frequencies. Can be changed to bias the starting values.
    #diag(freq)[pos.profile]<-f.profile
    
    
    #set up the Q matrix and use the dim.names as row and column names
    R<-matrix(r, dim, dim)
    rownames(R)<-colnames(R)<-dim.names

    #get the position of the states in the profile and outside the profile
    pos.profile<-match(profile, dim.names)
    pos.nonprofile<-which(is.na(match(dim.names,profile))==T)
    R[pos.profile,pos.nonprofile]<-s #rate of change from coevol to non-coevol
    R[pos.nonprofile,pos.profile]<-d #rate of change from non-coevol to coevol

    #finally, check if states differ by 2 states based on their names. 
    #  If yes, set the rate to 0
    for(i in 1:(length(dim.names)-1)) {
        for(j in (i+1):length(dim.names)) {
            x<-adist(dim.names[i], dim.names[j])
            if(x==2) {
                R[i,j]<-0
                R[j,i]<-0
            }
        }
    }

    if(withinProfile) {
        R[pos.profile,pos.profile]<-max(c(s,d)) #if you're in the profile, you don't change
    }
    
    #set the diagonal to 0
    diag(R)<-0
    #and then calculate the rowSums for the total amount of change.
    
    Q<-(R %*% freq) - diag(apply(R %*% freq, 1, sum))
    scaleQ <- sum(apply(freq %*% R %*% freq, 1, sum))

    return(list(Q=Q/scaleQ, scaleQ=scaleQ, dim.names=dim.names, profile=profile))
}


In [3]:
buildLG<-function(r=1, nStates=20) {
    
    dim<-nStates
    
    if(nStates == 4) {
        states<-c("A","C","G","T")
    }
    else if(nStates == 20) {
        states<-c("A","R","N","D","C","Q","E","G","H","I","L","K","M","F","P","S","T","W","Y","V")
    }
    else {
        states<-as.character(1:nStates)
    }
    
    f<-rgamma(dim, 100, 1) #almost equal frequency between states
    f<-f/sum(f)
    
    freq<-matrix(0, dim, dim)
    rownames(freq)<-colnames(freq)<-states
    
    diag(freq)<-f
    
    R<-matrix(r, dim, dim)
    rownames(R)<-colnames(R)<-states
    
    diag(R)<-0
    #and then calculate the rowSums for the total amount of change.
    
    Q<-(R %*% freq) - diag(apply(R %*% freq, 1, sum))
    scaleQ <- sum(apply(freq %*% R %*% freq, 1, sum))

    return(list(Q=Q/scaleQ, scaleQ=scaleQ, dim.names=states))
}

buildLG()

,A,R,N,D,C,Q,E,G,H,I,L,K,M,F,P,S,T,W,Y,V
A,-0.99146367,0.05200156,0.05183785,0.05932115,0.05027348,0.05124229,0.04951909,0.05633803,0.05090811,0.05291069,0.05395217,0.04849665,0.04726288,0.05167407,0.05603748,0.04992096,0.05614599,0.06033178,0.04817618,0.04511326
R,0.06154247,-1.00100458,0.05183785,0.05932115,0.05027348,0.05124229,0.04951909,0.05633803,0.05090811,0.05291069,0.05395217,0.04849665,0.04726288,0.05167407,0.05603748,0.04992096,0.05614599,0.06033178,0.04817618,0.04511326
N,0.06154247,0.05200156,-1.00116829,0.05932115,0.05027348,0.05124229,0.04951909,0.05633803,0.05090811,0.05291069,0.05395217,0.04849665,0.04726288,0.05167407,0.05603748,0.04992096,0.05614599,0.06033178,0.04817618,0.04511326
D,0.06154247,0.05200156,0.05183785,-0.99368498,0.05027348,0.05124229,0.04951909,0.05633803,0.05090811,0.05291069,0.05395217,0.04849665,0.04726288,0.05167407,0.05603748,0.04992096,0.05614599,0.06033178,0.04817618,0.04511326
C,0.06154247,0.05200156,0.05183785,0.05932115,-1.00273266,0.05124229,0.04951909,0.05633803,0.05090811,0.05291069,0.05395217,0.04849665,0.04726288,0.05167407,0.05603748,0.04992096,0.05614599,0.06033178,0.04817618,0.04511326
Q,0.06154247,0.05200156,0.05183785,0.05932115,0.05027348,-1.00176385,0.04951909,0.05633803,0.05090811,0.05291069,0.05395217,0.04849665,0.04726288,0.05167407,0.05603748,0.04992096,0.05614599,0.06033178,0.04817618,0.04511326
E,0.06154247,0.05200156,0.05183785,0.05932115,0.05027348,0.05124229,-1.00348704,0.05633803,0.05090811,0.05291069,0.05395217,0.04849665,0.04726288,0.05167407,0.05603748,0.04992096,0.05614599,0.06033178,0.04817618,0.04511326
G,0.06154247,0.05200156,0.05183785,0.05932115,0.05027348,0.05124229,0.04951909,-0.99666810,0.05090811,0.05291069,0.05395217,0.04849665,0.04726288,0.05167407,0.05603748,0.04992096,0.05614599,0.06033178,0.04817618,0.04511326
H,0.06154247,0.05200156,0.05183785,0.05932115,0.05027348,0.05124229,0.04951909,0.05633803,-1.00209802,0.05291069,0.05395217,0.04849665,0.04726288,0.05167407,0.05603748,0.04992096,0.05614599,0.06033178,0.04817618,0.04511326
I,0.06154247,0.05200156,0.05183785,0.05932115,0.05027348,0.05124229,0.04951909,0.05633803,0.05090811,-1.00009544,0.05395217,0.04849665,0.04726288,0.05167407,0.05603748,0.04992096,0.05614599,0.06033178,0.04817618,0.04511326


In [4]:
#s=0.1,d=1000,r=10 -> constant with profile
simulateCoev<-function(s=1, d=10, r=1, nStates=20, withinProfile=TRUE, nCoevol=25, nNonCoevol=500, nsp=100, meanBL=100, target=5) {
    tree<-pbtree(n=nsp, scale=1)
    tree$edge.length<-rexp(length(tree$edge.length), meanBL)
    
    sum.bl<-sum(tree$edge.length)
    tree$edge.length<-tree$edge.length*target/sum.bl
    
    logResults<-list()
    sequences<-rep("", nsp)
    treeCoev<-tree
    treeCoev$edge.length<-tree$edge.length * 2
    cat("Simulating coevolving positions: ")
    for(i in 1:nCoevol) {
        cat(".")
        model<-buildCoev(s=s,d=d,r=r,nStates=nStates,withinProfile=withinProfile)
        logResults[[i]]<-model$profile
        tt<-sim.history(treeCoev,model$Q,anc=sample(model$profile, 1),message=F)
        sequences<-paste(sequences, tt$states, sep="")
    }
    cat("\n\nSimulating non-coevolving positions: ")
    
    for(i in 1:nNonCoevol) {
        cat(".")
        model<-buildLG(r=r, nStates=nStates)
        tt<-sim.history(tree, model$Q, anc=sample(model$dim.names, 1),message=F)
        sequences<-paste(sequences, tt$states, sep="")
    }
    cat("\n\nDone.\n")
    names(sequences)<-names(tt$states)
    return(list(sequences=sequences, coevolProfiles=logResults, treeDepth=sum(tree$edge.length), tree=tree))
}

In [5]:
simulate_No_Coev<-function(s=1, d=10, r=1, nStates=20, withinProfile=TRUE, nNonCoevol=500, nsp=100, meanBL=100, target=5) {
  tree<-pbtree(n=nsp, scale=1)
  tree$edge.length<-rexp(length(tree$edge.length), meanBL)
  
  sum.bl<-sum(tree$edge.length)
  tree$edge.length<-tree$edge.length*target/sum.bl
  
  logResults<-list()
  sequences<-rep("", nsp)
  cat("\n\nSimulating non-coevolving positions: ")
  
  for(i in 1:nNonCoevol) {
    cat(".")
    model<-buildLG(r=r, nStates=nStates)
    tt<-sim.history(tree, model$Q, anc=sample(model$dim.names, 1),message=F)
    sequences<-paste(sequences, tt$states, sep="")
  }
  cat("\n\nDone.\n")
  names(sequences)<-names(tt$states)
  return(list(sequences=sequences, coevolProfiles=logResults, treeDepth=sum(tree$edge.length), tree=tree))
}


In [ ]:
for(i in 1:20) {
    x<-simulateCoev(s=1, d=20, r=1, nsp=50, nCoevol=10, nNonCoev=80, meanBL=10, target=10)
    write.FASTA(as.AAbin(as.matrix(x$sequences)), file=paste("coevol_test_", i, ".fasta", sep=""))
    write.tree(x$tree, file=paste("coevol_test_", i, ".tre", sep=""))
    cat(x[["treeDepth"]])
}
x$sequences
x$treeDepth
x$coevolProfiles
as.AAbin(as.matrix(x$sequences))
t1<-mergeMappedStates(tt,c("aa","ab"),"a")
t1<-mergeMappedStates(t1,c("ba","bb"),"b")
t2<-mergeMappedStates(tt,c("aa","ba"),"a")
t2<-mergeMappedStates(t2,c("ab","bb"),"b")
par(mfrow=c(1,1))
plotSimmap(t1,setNames(c("red","blue"),letters[1:2]),lwd=1,ftype="off")
plotSimmap(t2,setNames(c("red","blue"),letters[1:2]),lwd=1,ftype="off", direction="leftwards")


for(i in 1:20) {
  x<-simulateCoev(s=1, d=20, r=1, nsp=50, nCoevol=10, nNonCoev=80, meanBL=10, target=10)
  name_fasta <- paste('simulated_aa_v', i,"/scale_v10/fasta.txt", sep="")
  name_tree <- paste('simulated_aa_v', i,"/scale_v10/tree.txt", sep="")
  write.FASTA(as.AAbin(as.matrix(x$sequences)), file=name_fasta)
  write.tree(x$tree, file=name_tree)
  cat(x[["treeDepth"]])
}

In [ ]:
for(j in c(0.5,0.8,1,2,5,8,15,20,50,70)){
  for(i in 1:20) {
    x<-simulateCoev(s=1, d=20, r=1, nsp=50, nCoevol=10, nNonCoev=80, meanBL=10, target=j)
    name_fasta <- paste('simulated_aa_v', i,"/scale_v", j,"/fasta.txt", sep="")
    name_tree <- paste('simulated_aa_v', i,"/scale_v", j,"/tree.txt", sep="")
    write.FASTA(as.AAbin(as.matrix(x$sequences)), file=name_fasta)
    write.tree(x$tree, file=name_tree)
    cat(x[["treeDepth"]])
  }
  print("$$$$$$$$$$$$$$$$$$")
  print("$$$$$$$$$$$$$$$$$$")
  
  print(j)
  
  
  print("$$$$$$$$$$$$$$$$$$")
  print("$$$$$$$$$$$$$$$$$$")
  
}

In [8]:
path_data = '/Users/rramabal/Documents/PhD/Coevolution/Aminoacides/Simulated/CNN_project/'
path_data = paste(path_data,'DataSet_8/rawData/COEV/', sep="")

for (i in 1:500){
  
    num_coev = round(rnorm(1, mean=4, sd=1))
    
    while (num_coev == 0) {
      num_coev = round(rnorm(1, mean=4, sd=1))
    }
      x<-simulateCoev(s=1, d=20, r=1, nsp=40, nCoevol=num_coev, nNonCoev=20-(num_coev*2), meanBL=10, target=15)
      name_fasta <- paste('fasta_', i,'_coev_', num_coev,".txt", sep="")
      name_tree <- paste('tree_', i,'_coev_', num_coev,".txt", sep="")
      write.FASTA(as.AAbin(as.matrix(x$sequences)), file=paste(path_data, name_fasta, sep=""))
      write.tree(x$tree, file=paste(path_data, name_tree, sep=""))
      cat(x[["treeDepth"]])
}

Simulating coevolving positions: ...

Simulating non-coevolving positions: ..............

Done.
15Simulating coevolving positions: ....

Simulating non-coevolving positions: ............

Done.
15Simulating coevolving positions: ...

Simulating non-coevolving positions: ..............

Done.
15Simulating coevolving positions: ...

Simulating non-coevolving positions: ..............

Done.
15Simulating coevolving positions: ...

Simulating non-coevolving positions: ..............

Done.
15Simulating coevolving positions: ...

Simulating non-coevolving positions: ..............

Done.
15Simulating coevolving positions: ....

Simulating non-coevolving positions: ............

Done.
15Simulating coevolving positions: ...

Simulating non-coevolving positions: ..............

Done.
15Simulating coevolving positions: ...

Simulating non-coevolving positions: ..............

Done.
15Simulating coevolving positions: .....

Simulating non-coevolving positions: ..........

Done.
15Simulating coe

In [ ]:
path_data = '/Users/rramabal/Documents/PhD/Coevolution/Aminoacides/Simulated/CNN_project/'
path_data = paste(path_data,'DataSet_8/rawData/NO_COEV/',sep="")

for (i in 1:500){
  
  x<-simulate_No_Coev(s=1, d=20, r=1, nsp=40, nNonCoev=20, meanBL=10, target=15)
  name_fasta <- paste('fasta_', i,'_no_coev.txt', sep="")
  name_tree <- paste('tree_', i,'_no_coev.txt', sep="")
  write.FASTA(as.AAbin(as.matrix(x$sequences)), file=paste(path_data, name_fasta, sep=""))
  write.tree(x$tree, file=paste(path_data, name_tree, sep=""))
  cat(x[["treeDepth"]])
}